In [ ]:
import random
import numpy as np
from sklearn.metrics import *
import keras
from keras.layers import *
from keras.models import Model
from keras.optimizers import *
from keras import backend as K
from keras.utils.np_utils import to_categorical
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [ ]:
SHARD_NUM=10
SAMPLE_DATA_RATIO=0.2

In [ ]:
with open('adult.data','r')as f:
    traindata=[[p.strip() for  p in x.strip('\n').strip('.').split(',')] for x in f.readlines()]

with open('adult.test','r')as f:
    testdata=[[p.strip() for  p in x.strip('\n').strip('.').split(',')] for x in f.readlines()]

In [ ]:
datadict={i:{} for i in range(15)}

for data in traindata+testdata:
    for i in range(15):
        if data[i] not in datadict[i]:
            datadict[i][data[i]]=len(datadict[i])


In [ ]:
cateid=[1,3,5,6,7,8,9,13]
alldata=[]
alllabel=[]
for data in traindata+testdata:
    tp=[]
    for i in range(14):
        if i in cateid:
            tp+=to_categorical(datadict[i][data[i]],len(datadict[i])).tolist()
        else:
            tp.append(float(data[i]))
    
    alldata.append(tp)
    alllabel.append(datadict[14][data[14]])

In [ ]:
alldata=np.array(alldata)
alllabel=np.array(alllabel)
for i in range(len(alldata[0])):
    alldata[:,i]/=np.max(alldata[:,i])

In [ ]:

class DNN(Layer):
    def __init__(self, dim=200,**kwargs):
        #self.init = initializers.get('normal')
        self.dim = dim
        super(DNN, self).__init__(**kwargs)

    def build(self, input_shape):
        dim = self.dim
        self.W1 = tf.Variable(new_para_list[0],trainable=True)
        self.b1 = tf.Variable(new_para_list[1],trainable=True)
        self.W2 = tf.Variable(new_para_list[2],trainable=False)
        self.b2 = tf.Variable(new_para_list[3],trainable=False)        
        self.wa1 = self.add_weight(name='wa1',  shape=(new_para_list[0].shape[0],1,1),initializer='zeros', trainable=True)
        self.ba1 = self.add_weight(name='ba1',   shape=(new_para_list[0].shape[0],1), initializer='zeros',trainable=True)
        self.wa2 = self.add_weight(name='wa2',   shape=(new_para_list[0].shape[0],1,1),initializer='zeros',trainable=True)     
        self.ba2 = self.add_weight(name='ba2',   shape=(new_para_list[0].shape[0],1),initializer='zeros',trainable=True)     
 
        self.trainable_weights = [self.wa1,self.ba1,self.wa2,self.ba2]
        super(DNN,self).build(input_shape)  # be sure you call this somewhere!

    def call(self, key, mask=None, **kwargs):
        
        W1=K.sum(K.softmax(self.wa1,axis=0)*self.W1,axis=0)
        b1=K.sum(K.softmax(self.ba1,axis=0)*self.b1,axis=0)
        W2=K.sum(K.softmax(self.wa2,axis=0)*self.W2,axis=0)
        b2=K.sum(K.softmax(self.ba2,axis=0)*self.b2,axis=0)
        logit = K.dot(K.relu(K.dot(key, W1) + b1), W2)+b2
        logit = K.sigmoid(logit)
        
        return logit

    def compute_mask(self, input, input_mask=None):
        return None

    def compute_output_shape(self, input_shape):
        return input_shape[0], self.b2.shape[-1]


In [ ]:
all_acc=[]
all_pred=[]
all_para=[]

for i in range(SHARD_NUM):
    keras.backend.clear_session()
    feat_input = Input(shape=(108,), dtype='float32') 
    hidden=Dense(256,activation='relu')(feat_input)
    output=Dense(1,activation='sigmoid')(hidden)
    
    model = Model([feat_input], output)
    
    model.compile(loss=['binary_crossentropy'], optimizer=Adam(lr=0.01), metrics=['acc'])
    #for m in range(5):
    model.fit(alldata[len(traindata)//SHARD_NUM*i:len(traindata)//SHARD_NUM*(i+1)], alllabel[len(traindata)//SHARD_NUM*i:len(traindata)//SHARD_NUM*(i+1)],epochs=5,batch_size=64)
    pred=model.predict(alldata[len(traindata):]) 
    all_pred.append(pred)
    print(accuracy_score(alllabel[len(traindata):],np.round(pred)))
    all_acc.append(accuracy_score(alllabel[len(traindata):],np.round(pred)))
    all_para.append(model.get_weights()) 


In [ ]:

old_models=list(all_para)    
delete_shard_index= np.random.randint(0,SHARD_NUM-1)
delete_sample_index = np.random.randint(0,len(traindata)//SHARD_NUM-1)
updated_index=np.array([x for x in np.arange(len(traindata)//SHARD_NUM*delete_shard_index,len(traindata)//SHARD_NUM*(delete_shard_index+1)) if x!=(len(traindata)//SHARD_NUM*delete_shard_index+delete_sample_index)])

keras.backend.clear_session()
feat_input = Input(shape=(108,), dtype='float32') 
hidden=Dense(256,activation='relu')(feat_input)
output=Dense(1,activation='sigmoid')(hidden)

model = Model([feat_input], output) 
model.compile(loss=['binary_crossentropy'], optimizer=Adam(lr=0.01), metrics=['acc'])
model.fit(alldata[updated_index], alllabel[updated_index],epochs=5,batch_size=64)
all_para[delete_shard_index]=model.get_weights()
    
new_para_list=[np.array([x[i] for x in all_para]) for i in range(len(all_para[0]))]

In [ ]:
feat_input = Input(shape=(108,), dtype='float32') 
feat=DNN()(feat_input)
model_assemble = Model([feat_input], feat)
model_assemble.compile(loss=['binary_crossentropy'], optimizer=Adam(lr=0.05), metrics=['acc'])
indexer=np.array(random.sample(list(range(0,len(traindata)//SHARD_NUM*delete_shard_index+delete_sample_index))+ list(range(len(traindata)//SHARD_NUM*delete_shard_index+delete_sample_index+1,len(traindata))),int(len(traindata)//(SHARD_NUM/SAMPLE_DATA_RATIO))))
model_assemble.fit(alldata[indexer], alllabel[indexer],epochs=20,batch_size=64)
pred=model_assemble.predict(alldata[len(traindata):]) 
print(accuracy_score(alllabel[len(traindata):],np.round(pred)))
